In [125]:
!pip install pandas-datareader
import time
import numpy as np
import pandas as pd
import json
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import bs4 as bs
import pickle
import requests

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [126]:
def save_sp500_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

    soup = bs.BeautifulSoup(resp.text, "lxml")
    table = soup.find('table', {'class':'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        ticker = ticker[:-1]
        tickers.append(ticker)

    with open('sp500tickers.pickle','wb') as f:
        pickle.dump(tickers, f)
    return tickers

tickers = save_sp500_tickers()
tickers.sort()
tickers.remove('BF.B')
tickers.remove('BRK.B')
tickers.remove('CARR')
tickers.remove('DPZ')
tickers.remove('DXCM')
tickers.remove('OTIS')
tickers.remove('WST')

hm_days = 7

def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.0235
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0


In [127]:
from collections import OrderedDict
import pytz

data = OrderedDict()
stock_data_preprocessed = pd.DataFrame()
for ticker in tickers:
    df = pd.read_csv("stocks_dfs/{}.csv".format(ticker), index_col=0, parse_dates=['Date'])
    df = df[['Adj Close','Volume']]
    df.fillna(0, inplace=True)
    for i in range(1, hm_days+1):
        df['{}d'.format(i)] = (df['Adj Close'].shift(-i) - df['Adj Close']) / df['Adj Close']

    data[ticker] = df
    
stock_data_preprocessed = pd.concat(data.values(),keys=tickers,names=['Ticker','Date'])
stock_data_preprocessed = stock_data_preprocessed.swaplevel()
stock_data_preprocessed = stock_data_preprocessed.sort_index()

In [130]:
stock_data_preprocessed['target'] = list(map( buy_sell_hold,
                                               stock_data_preprocessed['1d'.format(ticker)],
                                               stock_data_preprocessed['2d'.format(ticker)],
                                               stock_data_preprocessed['3d'.format(ticker)],
                                               stock_data_preprocessed['4d'.format(ticker)],
                                               stock_data_preprocessed['5d'.format(ticker)],
                                               stock_data_preprocessed['6d'.format(ticker)],
                                               stock_data_preprocessed['7d'.format(ticker)] ))

stock_data_preprocessed.drop(columns=['1d', '2d','3d','4d','5d','6d','7d'], inplace=True)
stock_data_preprocessed.replace([np.inf, -np.inf], np.nan, inplace = True)
stock_data_preprocessed.fillna(0, inplace=True)

In [131]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
stock_data_preprocessed[['Adj Close', 'Volume']] = scaler.fit_transform(stock_data_preprocessed[['Adj Close', 'Volume']])
stock_data_preprocessed.to_csv('stock_data_preprocessed.csv')

In [176]:
date_start = pd.Timestamp('2019-01-01')
date_end = pd.Timestamp('2020-01-01')
AAPL = stock_data_preprocessed.loc[(slice(str(date_start),str(date_end)), 'AAPL'), :]

test_date_index = AAPL.index.droplevel(1)
test_date_index = pd.DataFrame(test_date_index)
test_date_index.to_csv('test_date_index.csv', index=False)

In [133]:
data_indicators = OrderedDict()
stock_data_indicators = pd.DataFrame()
for ticker in tickers:
    df = pd.read_csv("stocks_dfs/{}.csv".format(ticker), index_col=0, parse_dates=['Date'])
    df = df[['Adj Close','Volume']]
    df.fillna(0, inplace=True)
    for i in range(1, hm_days+1):
        df['{}d'.format(i)] = (df['Adj Close'].shift(-i) - df['Adj Close']) / df['Adj Close']
        
    df['50MA'] = df['Adj Close'].rolling(50).mean()
    df['25MA'] = df['Adj Close'].rolling(25).mean()
    df['10MA'] = df['Adj Close'].rolling(10).mean()
    df['5MA'] = df['Adj Close'].rolling(5).mean()

    df['50STD'] = df['Adj Close'].rolling(50).std()
    df['25STD'] = df['Adj Close'].rolling(25).std()
    df['10STD'] = df['Adj Close'].rolling(10).std()

    df['50UBB'] = df['50MA'] + (df['50STD'] * 2)
    df['25UBB'] = df['25MA'] + (df['25STD'] * 2)

    df['50LBB'] = df['50MA'] - (df['50STD'] * 2)
    df['25LBB'] = df['25MA'] - (df['25STD'] * 2)
    
    df['RSI5'] = get_rsi(5,df)
    df['RSI10'] = get_rsi(10,df)
    df['RSI25'] = get_rsi(25,df)
    df['RSI50'] = get_rsi(50,df)
    
    df['MACD5-10'] = get_macd(5,10,df)
    df['MACD5-25'] = get_macd(5,25,df)
    df['MACD5-50'] = get_macd(5,50,df)

    df['MACD10-25'] = get_macd(10,25,df)
    df['MACD10-50'] = get_macd(10,50,df)

    df['MACD25-50'] = get_macd(25,50,df)
    
    df['5DMomentum'] = get_n_day_momentum(5,df)
    df['10DMomentum'] = get_n_day_momentum(10,df)
    df['25DMomentum'] = get_n_day_momentum(25,df)
    df['50DMomentum'] = get_n_day_momentum(50,df)


    
    data[ticker] = df
    

stock_data_indicators = pd.concat(data.values(),keys=tickers,names=['Ticker','Date'])

stock_data_indicators = stock_data_indicators.swaplevel()
stock_data_indicators = stock_data_indicators.sort_index()

In [135]:
stock_data_indicators['target'] = list(map( buy_sell_hold,
                                               stock_data_indicators['1d'.format(ticker)],
                                               stock_data_indicators['2d'.format(ticker)],
                                               stock_data_indicators['3d'.format(ticker)],
                                               stock_data_indicators['4d'.format(ticker)],
                                               stock_data_indicators['5d'.format(ticker)],
                                               stock_data_indicators['6d'.format(ticker)],
                                               stock_data_indicators['7d'.format(ticker)] ))

In [143]:
#stock_data_indicators.drop(columns=['1d', '2d','3d','4d','5d','6d','7d'], inplace=True)
stock_data_indicators.replace([np.inf, -np.inf], np.nan, inplace = True)
stock_data_indicators.fillna(0, inplace=True)
stock_data_indicators = stock_data_indicators.loc[(slice('2010-03-16','2019-12-31')),:]
stock_data_indicators.head()

Adj Close       Volume       50MA       25MA       10MA  \
Date       Ticker                                                            
2010-03-16 A       21.820704    3285900.0  20.141775  20.705770  21.645719   
           AAL      7.325101    5915400.0   5.946059   6.822809   7.226114   
           AAP     41.161922     843400.0  39.806780  40.507213  40.935180   
           AAPL    27.833111  111727000.0  25.663924  25.941177  27.359656   
           ABC     24.367403    4210300.0  23.548246  23.958760  24.112215   

                         5MA     50STD     25STD     10STD      50UBB  ...  \
Date       Ticker                                                      ...   
2010-03-16 A       21.858573  0.961589  0.932832  0.303887  22.064953  ...   
           AAL      7.428802  0.970238  0.475839  0.240773   7.886535  ...   
           AAP     41.078227  1.027993  0.881267  0.261797  41.862767  ...   
           AAPL    27.907019  1.173133  1.326524  0.761078  28.010189  ...   
           ABC     24.087297  0.604464  0.298869  0.166858  24.757174  ...   

                   MACD5-25  MACD5-50  MACD10-25  MACD10-50  MACD25-50  \
Date       Ticker                                                        
2010-03-16 A       0.830135  1.242865   0.625629   1.038359   0.412730   
           AAL     0.574131  1.170651   0.457157   1.053676   0.596519   
           AAP     0.585189  0.970875   0.411384   0.797070   0.385686   
           AAPL    1.321352  1.690658   0.896776   1.266081   0.369306   
           ABC     0.178486  0.452716   0.154857   0.429087   0.274230   

                   5DMomentum  10DMomentum  25DMomentum  50DMomentum  target  
Date       Ticker                                                             
2010-03-16 A         0.097935     0.692101     2.755337          0.0       1  
           AAL       0.056563     0.084847     1.734644          0.0      -1  
           AAP       0.350330     0.934185     1.449917          0.0       1  
           AAPL      0.177330     1.934494     3.761101          0.0       0  
           ABC       0.317907    -0.085918     1.279705          0.0       1  

[5 rows x 28 columns]

In [148]:
from sklearn.preprocessing import StandardScaler

stock_data_indicators = stock_data_indicators.loc[(slice('2010-03-16','2019-12-31')),:]
indicators = stock_data_indicators.drop(columns=['Adj Close','Volume','target'])
indicators = indicators.values
indicators = StandardScaler().fit_transform(indicators)


In [170]:
from sklearn.decomposition import PCA
pca = PCA(.95)
principalComponents = pca.fit_transform(indicators)

In [173]:
indicator_index = stock_data_indicators.index

In [174]:
principalDf = pd.DataFrame(data = principalComponents, columns = ['PC1', 'PC2','PC3','PC4','PC5','PC6'], index = indicator_index)
principalDf.head()

PC1       PC2       PC3       PC4       PC5       PC6
Date       Ticker                                                            
2010-03-16 A      -0.752058 -1.622765  1.379154  0.829721  0.766182 -0.299314
           AAL    -1.155141 -1.345847  0.508184  0.746179  1.022953 -0.384040
           AAP    -0.637966 -0.966827  0.851218  0.315813  0.047650 -0.163081
           AAPL   -0.503010 -1.684244  1.363548  0.603027  0.750602 -0.149463
           ABC    -1.058094 -0.997907  0.891929  0.373091 -0.156564 -0.012736

In [169]:
#stock_indicator_data = stock_data_preprocessed.loc[(slice('2010-03-16','2019-12-31')),:]
#stock_indicator_data['PC1'] = principalDf['PC1']
#stock_indicator_data['PC2'] = principalDf['PC2']
#stock_indicator_data['PC3'] = principalDf['PC3']
#stock_indicator_data['PC4'] = principalDf['PC4']
#stock_indicator_data['PC5'] = principalDf['PC5']
#stock_indicator_data['PC6'] = principalDf['PC6']

#stock_indicator_data = stock_indicator_data.reindex(columns= ['Adj Close', 'Volume', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6','target'])

In [168]:
stock_indicator_data.to_csv('stock_indicator_data.csv')

In [175]:
date_start = pd.Timestamp('2010-03-16')
date_end = pd.Timestamp('2020-01-01')
AAPL = stock_data_preprocessed.loc[(slice(str(date_start),str(date_end)), 'AAPL'), :]

indicator_date_index = AAPL.index.droplevel(1)
indicator_date_index = pd.DataFrame(indicator_date_index)
indicator_date_index.to_csv('indicator_date_index.csv', index=False)

In [ ]:

def get_rsi(window_length, data):
    df = data
    delta = df['Adj Close'].diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    roll_up1 = up.ewm(span=window_length).mean()
    roll_down1 = down.abs().ewm(span=window_length).mean()
    RS = roll_up1 / roll_down1
    RSI = 100.0 - (100.0 / (1.0 + RS))
    return RSI

#get_rsi(5,stock_data_indicators)

In [ ]:
def get_macd(ewm1, ewm2, data):
    df = data
    exp1 = df['Adj Close'].ewm(span=ewm1, adjust=False).mean()
    exp2 =  df['Adj Close'].ewm(span=ewm2, adjust=False).mean()
    return exp1-exp2

#get_macd(12,26,df)

In [ ]:
def get_n_day_momentum(n_days, data):
    close = data['Adj Close']
    return close - close.shift(n_days)

get_n_day_momentum(20,AAPL)